![](images/Callysto_Notebook-Banner_Top.jpg)

---

In [3]:
!pip install onc --user --quiet
!pip install aiohttp --user --quiet
!pip install asyncio --user --quiet
!pip install --upgrade --force-reinstall --user --quiet git+git://github.com/callysto/nbplus.git@staging#egg=nbplus

aiohttp requires Python '>=3.4.2' but the running Python is 2.7.13


In [10]:
import datetime
import functools
import itertools
import urllib3
import asyncio
import nbvis.magics

from math import sqrt
from heapq import nlargest
from nbvis.classes import D3, Vis
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from IPython.display import Javascript
from json import dumps
from onc.onc import ONC

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

<h1 align="center">$\text{Oceans} \ \mathbb{2.0}$</h1>

<h3 align="center">Exploring Ocean Thermal Energy Conversion through Device Discovery</h3>

---

## Introduction

**Ocean thermal energy conversion** (OTEC) is a renewable energy resource that utilizes temperature differences to generate energy. Deeper, _cooler_ ocean waters function as a heat sink, also known as a <span style="color:blue ">cold</span> reservoir, with respect to shallower, _warmer_ surface waters, i.e. a <span style="color:red">hot</span> reservoir. If the temperature difference

$$\Delta T = T_{hot} - T_{cold}$$

between these reservoirs is substantial, the resulting flow of heat does mechanical work. Such a system is known as a **heat engine**.

Pelc and Fujita (2002) estimate that 10 trillion watts of power, approximately equal to the current global energy demand, could be provided by OTEC without affecting the thermal structure of the ocean. <sup>[0](#fn0)</sup>

We will leverage the **Oceans 2.0** application programming interface (API) made available by Ocean Networks Canada (ONC) to seek candidate sites for this technology. To do so, we will gather temperature and location data from numerous sensors along the North American coastline, and quantify hypothetically the maximal efficiency of a heat engine assembled  between any two of these locations.

---

## Background

#### Quantifying Efficiency

How efficient is the conversion of thermal energy to mechanical work?

That is, given the heat _input_ $Q_{heat}$ and mechanical _output_ $W$, what is the efficiency $\displaystyle\eta = {W\over Q_{heat}}$?

To quantify this, we consider the **Carnot heat engine**, an ideal and theoretical thermodynamic system. Its corresponding thermodynamic cycle, the **Carnot cycle**, places an upper limit on the efficiency of any classical heat engine. It turns out, by Carnot's theorem, that any heat engine between two reservoirs is less efficient than the Carnot heat engine. This is a result of the second law of thermodynamics. <sup>[1](#fn1)</sup>

Given two reservoir temperatures $T_{hot}$ and $T_{cold}$, using the Kelvin scale, we may compute this maximal efficiency

$$\eta_{Carnot} = 1 - {T_{cold} \over T_{hot}}.$$

Let us define a function for computing $\eta_{Carnot}$.

In [11]:
def max_efficiency(T_cold, T_hot): return 1 - T_cold / T_hot;

Let us also define a function for converting temperatures from the Celsius scale to the Kelvin scale. 

In [12]:
def celsius_to_kelvin(T): return T + 273.15;

We can now, for example, compute the maximal efficiency of a heat engine assembled between a <span style="color:red">hot</span> reservoir of boiling water, and a <span style="color:blue">cold</span> reservoir of ice.

In [13]:
print('%.2f' % round(max_efficiency(T_cold=celsius_to_kelvin(0), T_hot=celsius_to_kelvin(100)) * 100, 2), '%')

26.80 %


#### Querying Oceans 2.0

Oceans 2.0 is an online data management system courtesy of ONC, offering "ocean analytics that assist researchers, communities, industry, and policy-makers in making evidence-based decisions in Canada and globally." <sup>[2](#fn2)</sup> Accessing the Oceans 2.0 API is programmatic and available through a Python client library. The data is served three ways: <sup>[3](#fn3)</sup>

* **Discovery**:  we can find and filter device codes by some criteria.
* **Data Delivery**: once we have a device code, we can request data.
* **Interoperability**: we may want to share with or integrate other data sources.

We create an instance of `ONC()` class. This class wraps the methods we will use to gather data. It also requires an access token, which we load from a separate file.

In [14]:
onc = ONC(token=open('oceans2_api_token.txt', 'r').read())

Our search for candidate OTEC sites relies on temperature data. Oceans 2.0 considers this "scalar data," which is available through a class method `getDirectScalar()`. This method expects, at the least, a `locationCode` and a `deviceCategoryCode`. Two other class methods, `getDeviceCategories()` and `getLocations()`, will yield these codes. <sup>[4](#fn4)</sup>

#### Finding Properties

There may also be more than one way to observe temperature. We use `getProperties()` to see which phenomena are quantified, and produce a list of unique property codes via `set()` which include the substring `'temperature'`.

In [15]:
properties = onc.getProperties(filters={})
propertyCodes = list(set(p['propertyCode'] for p in properties)) 
[p for p in propertyCodes if 'temperature' in p]

['seawatertemperature',
 'windchilltemperature',
 'airtemperature',
 'srslicebalancebuoytemperature',
 'sedimenttemperature',
 'seaicetemperature',
 'wetbulbtemperature',
 'sedimentsurfacetemperature',
 'internaltemperature',
 'differentialtemperature',
 'ventfluidtemperature']

#### Finding Device Categories

We focus on the property code `'seawatertemperature'`.

In [16]:
propertyCode = 'seawatertemperature'
unitOfMeasure = 'C'

Let us gather the device category codes first via `getDeviceCategories()`, filtering by this property code.

In [17]:
categories = onc.getDeviceCategories(filters={'propertyCode': propertyCode})

Each element of `categories` has a `deviceCategoryCode` attribute. These are what we are after.

In [18]:
categoryCodes = [c['deviceCategoryCode'] for c in categories]
categoryCodes

['ADCP1200KHZ',
 'ADCP150KHZ',
 'ADCP1MHZ',
 'ADCP2MHZ',
 'ADCP300KHZ',
 'ADCP400KHZ',
 'ADCP55KHZ',
 'ADCP600KHZ',
 'ADCP75KHZ',
 'BBES',
 'BPR',
 'CO2SENSOR',
 'CORK',
 'CTD',
 'CURRENTMETER',
 'DEPTH_TEMP',
 'DIVE_COMPUTER',
 'ECHOSOUNDERBIOA',
 'GTD',
 'ICEPROFILER',
 'MBPROFILESONAR',
 'METHSENSOR',
 'NODE',
 'OXYSENSOR',
 'PHSENSOR',
 'PRES',
 'PYRANOMETER',
 'PYRGEOMETER',
 'TARRAY',
 'TEMPSENSOR',
 'TILTMTR',
 'TSG',
 'WETLABS_WQM']

#### Finding Locations

What remains is to determine where these devices are. The class method `getLocations()` can filter by device category code. So that we are convinced that these are the right locations, we also filter by the same property code.

Later we will get the locations of every device from every category that measures temperature. For now, we focus on the category code `'CTD'`. These are instruments that measure conductivity, temperature and depth.

In [19]:
categoryCode = 'CTD'
locations = onc.getLocations(filters = {
    'propertyCode': propertyCode,
    'deviceCategoryCode': categoryCode
})

As with the device categories, each element of `locations` has a `locationCode` attribute. We slice the first five for a preview.

In [20]:
locationCodes = [l['locationCode'] for l in locations]
locationCodes[:5]

['AS04', 'BACAX', 'BACCH', 'BACHY', 'BACME']

#### Getting Scalar Data

At last, we have what we need to request a data product from Oceans 2.0. Let us focus on the first location code, `'AS04'`.

In [21]:
firstLocationCode = locationCodes[0]
firstLocationCode

'AS04'

Let us also expect data no more than a day old, so that we query devices which are still deployed. We get the current time minus one day.

In [22]:
timeShift = datetime.timedelta(days=-1)
dateFrom = (datetime.datetime.now() + timeShift).isoformat('T')[:-3] + 'Z'
dateFrom

'2018-07-11T02:45:40.994Z'

Now we call to the class method `getDirectScalar()`, passing our sought after location and device category codes as arguments. Specifying `'metadata'` to be `'full'` requests the latitude and longitude of the location. The `rowLimit` restricts our request for data to at most five points in time.

In [23]:
scalarData = onc.getDirectScalar(filters = {
    'locationCode': firstLocationCode,
    'metadata': 'full',
    'deviceCategoryCode': categoryCode,
    'dateFrom': dateFrom,
    'rowLimit': 5
})
scalarData

{'metadata': {'boundingBox': {'maxDepth': 120.0,
   'maxLat': 48.300767,
   'maxLon': -123.390117,
   'minDepth': 120.0,
   'minLat': 48.300767,
   'minLon': -123.390117},
  'depth': 120.0,
  'deviceCategoryCode': 'CTD',
  'lat': 48.300767,
  'locationName': 'AS04 Mooring',
  'lon': -123.390117},
 'next': None,
 'queryUrl': 'https://data.oceannetworks.ca/api/scalardata?locationCode=AS04&metadata=full&deviceCategoryCode=CTD&dateFrom=2018-07-11T02%3A45%3A40.994Z&rowLimit=5&method=getByLocation&token=0b8544ec-977b-404a-a0b3-8e99216aeed6',
 'sensorData': None}

We check if there is sensor data, and if the latitude and longitude metadata was delivered.

If these exist, we look for a sensor with degrees Celsius as its unit of measure, and get its most recent data point. If not, we try other codes until we find one with data.

In [24]:
sensorData = scalarData['sensorData']
boundingBox = scalarData['metadata']['boundingBox']
if sensorData is not None and not any(v is None for v in list(boundingBox.values())):
    for s in sensorData:
        if s['unitOfMeasure'] == unitOfMeasure:
            sampleTime = s['data']['sampleTimes'][-1]
            value = s['data']['values'][-1]
            print(locationCode, '/', sampleTime, '/' , value, unitOfMeasure)
else:
    for locationCode in locationCodes:
        scalarData = onc.getDirectScalar(filters = {
            'locationCode': locationCode,
            'metadata': 'full',
            'deviceCategoryCode': categoryCode,
            'dateFrom': dateFrom,
            'rowLimit': 5
        })
        sensorData = scalarData['sensorData']
        boundingBox = scalarData['metadata']['boundingBox']
        if sensorData is not None and not any(v is None for v in list(boundingBox.values())):
            for s in sensorData:
                if s['unitOfMeasure'] == unitOfMeasure:
                    sampleTime = s['data']['sampleTimes'][-1]
                    value = s['data']['values'][-1]
                    print(locationCode, '/', sampleTime, '/' , value, unitOfMeasure)
            break;

BACAX / 2018-07-11T02:45:45.786Z / 3.8344 C


We may also gather the sensors' geographic coordinates, as well as its depth (in metres).

In [25]:
lat = (boundingBox['maxLat'] + boundingBox['minLat'])/2
lon = (boundingBox['maxLon'] + boundingBox['minLon'])/2
depth = (boundingBox['maxDepth'] + boundingBox['minDepth'])/2
print(lat, '/', lon, '/', depth)

48.316583 / -126.05076 / 983.0


---

## Demonstration

At this point we have isolated the temperature and location data of a single sensor. To determine candidate sites for OTEC, we should gather this data from many sensors from many locations.

Here we set up a list of coroutines to asynchronously request this data from every device category code we found before. These requests are made simultaneously, i.e. _in parallel_, as the result of one does not affect another. We gather the same data from each location, and pass these locations to D3.js for visualization. 

In [26]:
import asyncio
import concurrent.futures
import requests
import aiohttp

LOOP = asyncio.new_event_loop()
concurrent.futures.ThreadPoolExecutor().submit(LOOP.run_forever);

In [27]:
def getLocationsFromCategory(categoryCode, propertyCode):
    c = categoryCode; p = propertyCode
    locations = onc.getLocations(filters={
        'deviceCategoryCode': c,
        'propertyCode': p
    })
    locationCodes = list(map(lambda l: l['locationCode'], locations))
    return [(c, l) for l in locationCodes];

In [28]:
async def runInParallel(func, iterative, *args):
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(iterative)) as executor:
        futures = [
            LOOP.run_in_executor(executor, func, *i, *args)
                if isinstance(i, tuple)
                else LOOP.run_in_executor(executor, func, i, *args)
                    for i in iterative
        ]
        return await asyncio.gather(*futures)

result = asyncio.run_coroutine_threadsafe(runInParallel(getLocationsFromCategory, categoryCodes, propertyCode), LOOP).result()
categoryLocationId = [item for sublist in result for item in sublist]

In [29]:
def getPropertyData(data, propertyCode, unitOfMeasure):
    if data != []:
        sensorData = data['sensorData']
        for i in sensorData:
            if i['unitOfMeasure'] == unitOfMeasure:
                val = i['data']['values'][0]
                time = i['data']['sampleTimes'][0]
                unit = i['unitOfMeasure']
                return time, val, unit;
    else: return [];

def getCoordinates(data):
    lat, lon, depth = None, None, None
    if data != []:
        try:
            boundingBox = data['metadata']['boundingBox']
            lat = (boundingBox['maxLat'] + boundingBox['minLat'])/2
            lon = (boundingBox['maxLon'] + boundingBox['minLon'])/2
            depth = (boundingBox['maxDepth'] + boundingBox['minDepth'])/2
        except TypeError: pass
    return lat, lon, depth;

def getDataFromLocation(categoryCode, locationCode, dateFrom):
    data = onc.getDirectScalar(filters = {
        'locationCode': locationCode,
        'metadata': 'full',
        'deviceCategoryCode': categoryCode,
        'dateFrom': dateFrom,
        'rowLimit': 1
    })
    if data['sensorData'] is not None:
        temperature = getPropertyData(data, propertyCode, unitOfMeasure)
        if temperature is not None:
            coordinates = getCoordinates(data)
            if None not in coordinates:
                datum = {
                    "categoryCode": categoryCode,
                    "locationCode": locationCode,
                    "temperature": temperature[1],
                    "depth": coordinates[2],
                    "latitude": coordinates[1],
                    "longitude": coordinates[0]
                }
                display(Javascript("data.push(%s); updateCircles();" % datum))
                display(Javascript("console.log(%s);" % datum))
                return datum
    else:
        display(Javascript("console.log('No sensor data from %s at %s');" % (categoryCode, locationCode)))

In [30]:
%%d3 --reset --queue
window.data = [];
var svg = d3.select("svg#geography");

svg.selectAll("*").remove();
var width = +svg.node().getBoundingClientRect().width;
var height = +svg.attr("height");

const projection = d3.geoMercator();
const initialScale = projection.scale();
projection
    .scale(850)
    .rotate([100, -60, 0])
    .translate([width/2, height/2]);

const geoPath = d3.geoPath().projection(projection);
const path = svg.append('path').attr('stroke', 'gray');
const circlePath =  d3.geoPath().projection(projection);

var graticule = d3.geoGraticule()

svg.append("path")
    .datum(graticule)
    .attr("class", "graticule")
    .style("fill", "none")
    .style("stroke", "#777")
    .style("stroke-width", "0.5px")
    .style("stroke-opacity", "0.5")
    .attr("d", geoPath);

let moving = false;

var geojson = [
    "https://unpkg.com/world-atlas@1/world/110m.json",
    "https://unpkg.com/world-atlas@1/world/50m.json"
];

window.updateCircles = () => {
    svg.append("g")
               .selectAll("path")
               .data(data)
               .enter().append("path")
               .attr("fill", "red")
               .attr("d", d => geoPath(d3.geoCircle()
                                         .center([d.latitude, d.longitude])
                                         .radius(2e-1/d3.zoomTransform(svg.node()).k)()))
}

Promise.all(geojson.map(url => d3.json(url)))
    .then(function([world110m, world50m]) {

        const countries110m = topojson
            .feature(world110m, world110m.objects.countries);
        const countries50m = topojson
            .feature(world50m, world50m.objects.countries);
        const render = () => {
            svg.selectAll("path").attr("d", geoPath)

            updateCircles();

            // render low/high resolution boundaries when moving/stopped
            path.attr('d', geoPath(moving ? countries110m : countries50m));
        };
        render();

        let rotate0, coords0;
        const coords = () => projection.rotate(rotate0)
            .invert([d3.event.x, d3.event.y]);

        svg
            .call(d3.drag()
                .on('start', () => {
                    rotate0 = projection.rotate();
                    coords0 = coords();
                    moving = true;
                })
                .on('drag', () => {
                    const coords1 = coords();
                    projection.rotate([
                                  rotate0[0] + coords1[0] - coords0[0],
                                  rotate0[1] + coords1[1] - coords0[1],
                    ])
                    render();
                })
                .on('end', () => {
                  moving = false;
                  render();
                })
              )

          d3.geoZoom()
            .projection(projection)
            .northUp(true)
            .onMove(render)
          (svg.node());
    })
    .catch(e => { console.log(e); }); 

Initialized d3_code container!
Code added to D3 visualization queue ...


<IPython.core.display.Javascript object>

In [31]:
geography = D3("geography").svg(height=600)
geography.require("d3-geo-projection", "d3-geo-zoom")
Vis(geography);

<IPython.core.display.Javascript object>

In [32]:
data = asyncio.run_coroutine_threadsafe(runInParallel(getDataFromLocation, categoryLocationId, dateFrom), LOOP).result();

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

## Optimization

Initially we set out to find candidate sites for OTEC. In our demonstration above, we patiently recover `data`, which contains the temperature and location data for every sensor that Oceans 2.0 could deliver.

In [33]:
data[:5]

[{'categoryCode': 'ADCP1200KHZ',
  'locationCode': 'CBYIP',
  'temperature': 0.45,
  'depth': 9.0,
  'latitude': -105.06403,
  'longitude': 69.1129},
 {'categoryCode': 'ADCP150KHZ',
  'locationCode': 'BACME',
  'temperature': 3.97,
  'depth': 895.0,
  'latitude': -126.058533,
  'longitude': 48.314901},
 None,
 {'categoryCode': 'ADCP150KHZ',
  'locationCode': 'SCVIP',
  'temperature': 9.13,
  'depth': 298.0,
  'latitude': -123.425809,
  'longitude': 49.040683},
 {'categoryCode': 'ADCP150KHZ',
  'locationCode': 'SEVIP',
  'temperature': 9.73,
  'depth': 166.0,
  'latitude': -123.316579,
  'longitude': 49.042586}]

With this we may compute the ideal locations for <span style="color:blue">cold</span> and <span style="color:red">hot</span> reservoirs to maximize the effiency of a Carnot heat engine.

As we see on the map, some locations are simply too far apart for OTEC. We should only consider locations that are near one another.

To constrain our results, we create a function, `nearby(latitudeDifference, longitudeDifference, depthDifference)`, to check if the Euclidean distance between any two locations is less than 2 kilometres. We also check that the distance is greater than zero, to avoid a physical impossibility. As an approximation, we suppose that a degree of latitude or longitude is at most 112 kilometres. <sup>[5](#fn5)</sup> 

In [34]:
def nearby(latitudeDifference, longitudeDifference, depthDifference):
    distance = sqrt(latitudeDifference*latitudeDifference + longitudeDifference*longitudeDifference + depthDifference*depthDifference)
    if 0 < distance < 2:
        return True;
    return False;

We generate a list of all possible pairs of sensors, and slice the first two for a preview.

In [35]:
sensorPairs = list(itertools.combinations(data, 2))
sensorPairs[:2]

[({'categoryCode': 'ADCP1200KHZ',
   'locationCode': 'CBYIP',
   'temperature': 0.45,
   'depth': 9.0,
   'latitude': -105.06403,
   'longitude': 69.1129},
  {'categoryCode': 'ADCP150KHZ',
   'locationCode': 'BACME',
   'temperature': 3.97,
   'depth': 895.0,
   'latitude': -126.058533,
   'longitude': 48.314901}),
 ({'categoryCode': 'ADCP1200KHZ',
   'locationCode': 'CBYIP',
   'temperature': 0.45,
   'depth': 9.0,
   'latitude': -105.06403,
   'longitude': 69.1129},
  None)]

Next we filter `sensorPairs`, overlooking sensors that are mutually more than 2 kilometres apart.

In [36]:
closeSensorPairs = []
for pair in sensorPairs:
    try:
        latitudeDifference = (pair[0]['latitude'] - pair[1]['latitude']) * 112
        longitudeDifference = (pair[0]['longitude'] - pair[1]['longitude']) * 112
        depthDifference = (pair[0]['depth'] - pair[1]['depth']) / 1000
    
        if nearby(latitudeDifference, longitudeDifference, depthDifference):
            closeSensorPairs += [pair]
    except TypeError: pass
closeSensorPairs[:2]

[({'categoryCode': 'ADCP150KHZ',
   'locationCode': 'BACME',
   'temperature': 3.97,
   'depth': 895.0,
   'latitude': -126.058533,
   'longitude': 48.314901},
  {'categoryCode': 'ADCP2MHZ',
   'locationCode': 'BACME',
   'temperature': 4.14,
   'depth': 893.0,
   'latitude': -126.058587,
   'longitude': 48.31494}),
 ({'categoryCode': 'ADCP150KHZ',
   'locationCode': 'BACME',
   'temperature': 3.97,
   'depth': 895.0,
   'latitude': -126.058533,
   'longitude': 48.314901},
  {'categoryCode': 'ADCP55KHZ',
   'locationCode': 'BACAX',
   'temperature': 4.95,
   'depth': 981.0,
   'latitude': -126.050718,
   'longitude': 48.31664})]

From `closeSensorPairs`, we generate a list of all possible pairs of locations, as well as a list of all possible pairs of temperatures $\left(T_{cold}, T_{hot}\right)$, with every temperature represented using the Kelvin scale.

We `zip` these lists to preserve the pairing of locations with temperatures. Naturally, the <span style="color:blue">cold</span> reservoir will be one of two locations with a lower temperature, so we ignore elements where $T_{cold}$ is greater than $T_{hot}$.

In theory, OTEC provides as much energy with a 20 degree temperature difference as a hydroelectric plant with 34 metre head, given the same volume of water flow. <sup>[6](#fn6)</sup>

It is unreasonable to expect such a temperature difference $\Delta T = T_{hot} - T_{cold}$ between nearby locations, namely this from the equator. In the event that there is no two locations with a temperature difference greater than 20 degrees, we check again for elements with a temperature difference of 19 degrees, then 18 degrees, and so forth, until we find a suitable location.

In [37]:
temperatureDifference = 20
locationPairs = [(c[0]['locationCode'], c[1]['locationCode']) for c in closeSensorPairs]
temperaturePairs = [(celsius_to_kelvin(c[0]['temperature']), celsius_to_kelvin(c[1]['temperature'])) for c in closeSensorPairs]
locationTemperaturePairs = [pair for pair in list(zip(locationPairs, temperaturePairs)) if pair[1][0] < pair[1][1] and not abs(pair[1][1] - pair[1][0]) < temperatureDifference]
while locationTemperaturePairs == [] :
    temperatureDifference -= 1
    locationTemperaturePairs = [pair for pair in list(zip(locationPairs, temperaturePairs)) if pair[1][0] < pair[1][1] and not abs(pair[1][1] - pair[1][0]) < temperatureDifference]
locationTemperaturePairs

[(('KEMF', 'MEFS'), (275.03, 286.9)),
 (('RCSE5.A1', 'MEFS'), (275.21, 286.9)),
 (('RCSE5.A2', 'MEFS'), (275.34999999999997, 286.9))]

These pairs of temperatures are readily unpacked and passed to our function, `max_efficiency(T_cold, T_hot)`, that we defined at the onset. We isolate the temperature pair that maximizes the efficiency of a Carnot heat engine. So that we may determine the locations of its heat reservoirs, we get the indices of `locationTemperaturePairs` where the greatest efficiency values are attained.

In [38]:
efficiencies = [round(max_efficiency(*pair[1]) * 100, 2) for pair in locationTemperaturePairs] 
indices = [efficiencies.index(efficiency) for efficiency in nlargest(3, efficiencies)]
list(zip(['%.2f ' % efficiencies[i] + '%' for i in indices], indices))

[('4.14 %', 0), ('4.07 %', 1), ('4.03 %', 2)]

Finally, we may isolate the candidate sites for OTEC.

In [39]:
candidateSites = [locationTemperaturePairs[i][0] for i in indices]
candidateSites

[('KEMF', 'MEFS'), ('RCSE5.A1', 'MEFS'), ('RCSE5.A2', 'MEFS')]

We use `getLocations()` once more with each location code to name these locations and determine their depths.

In [40]:
for sites in candidateSites:
    coldReservoir = onc.getLocations(filters = { 'locationCode': sites[0] })[0]
    hotReservoir = onc.getLocations(filters = { 'locationCode': sites[1] })[0]
    coldReservoirInfo = coldReservoir['locationCode'] + ' / ' + coldReservoir['locationName'] + ' / ' + str(coldReservoir['depth'])
    hotReservoirInfo = hotReservoir['locationCode'] + ' / ' + hotReservoir['locationName'] + ' / ' + str(hotReservoir['depth'])
    print(coldReservoirInfo, ' to ', hotReservoirInfo)

KEMF / Main Endeavour Field / 2191.226087  to  MEFS / Main Endeavour Field South / 2188.333333
RCSE5.A1 / ADCP Upward / 1977.0  to  MEFS / Main Endeavour Field South / 2188.333333
RCSE5.A2 / ADCP Downward / 1977.0  to  MEFS / Main Endeavour Field South / 2188.333333


These are candidate sites for OTEC!

---

## Conclusion

We set out to find optimal development sites along Canadian coastlines for Ocean Thermal Energy Conversion, a hypothetical and renewable energy resource that, acting as heat engine, utilizes temperature differences to generate energy. We simplified our exploration by considering the Carnot heat engine, a theoretical physical model, to place an upper bound on the efficiency of OTEC developed between two sites.

The use of the Oceans 2.0 API, courtesy of Ocean Networks Canada, allowed us to find all temperature sensors catalogued by ONC. With this temperature data, we narrowed our search to _nearby_ candidate sites with the greatest temperature difference, and computed the maximal efficiency $\eta_{Carnot}$ of OTEC at each site.

---

#### Citations

<a name="fn0">0</a> : https://centerforoceansolutions.org/sites/default/files/publications/Pelc%20and%20Fujita%202002.pdf

<a name="fn1">1</a> : https://en.wikipedia.org/wiki/Carnot%27s_theorem_(thermodynamics)

<a name="fn2">2</a> : https://www.oceannetworks.ca

<a name="fn3">3</a> : https://wiki.oceannetworks.ca/display/O2A/API+Reference

<a name="fn4">4</a> : https://wiki.oceannetworks.ca/display/O2A/Python+Client+Library

<a name="fn5">5</a> : https://gis.stackexchange.com/a/142327

<a name="fn6">6</a> : https://en.wikipedia.org/wiki/Ocean_thermal_energy_conversion#Thermodynamics

---

![](images/Callysto_Notebook-Banners_Bottom.jpg)